# Periodic download of IEX stock-ticker data

## Read IEX API credentials from `~/.config/iex.ini`

In [2]:
from pathlib import Path
config_path = Path.home() / '.config' / 'iex.ini'

from configparser import ConfigParser
config = ConfigParser()
config.read(str(config_path))
iex_config = config['iex']

api = 'https://cloud.iexapis.com'
public_key = iex_config['public_key']
secret_key = iex_config['secret_key']

In [3]:
tickers = sorted("MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AMG AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE AGN ADS LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AIV AAPL AMAT APTV ADM ARNC ANET AJG AIZ ATO T ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BBT BDX BRK.B BBY BIIB BLK HRB BA BKNG BWA BXP BSX BMY AVGO BR BF.B CHRW COG CDNS CPB COF CPRI CAH KMX CCL CAT CBOE CBRE CBS CDW CE CELG CNC CNP CTL CERN CF SCHW CHTR CVX CMG CB CHD CI XEC CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG CXO COP ED STZ COO CPRT GLW CTVA COST COTY CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN FANG DLR DFS DISCA DISCK DISH DG DLTR D DOV DOW DTE DUK DRE DD DXC ETFC EMN ETN EBAY ECL EIX EW EA EMR ETR EOG EFX EQIX EQR ESS EL EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FLS FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GD GE GIS GM GPC GILD GL GPN GS GWW HAL HBI HOG HIG HAS HCA HCP HP HSIC HSY HES HPE HLT HFC HOLX HD HON HRL HST HPQ HUM HBAN HII IEX IDXX INFO ITW ILMN IR INTC ICE IBM INCY IP IPG IFF INTU ISRG IVZ IPGP IQV IRM JKHY JEC JBHT SJM JNJ JCI JPM JNPR KSU K KEY KEYS KMB KIM KMI KLAC KSS KHC KR LB LHX LH LRCX LW LVS LEG LDOS LEN LLY LNC LIN LKQ LMT L LOW LYB MTB MAC M MRO MPC MKTX MAR MMC MLM MAS MA MKC MXIM MCD MCK MDT MRK MET MTD MGM MCHP MU MSFT MAA MHK TAP MDLZ MNST MCO MS MOS MSI MSCI MYL NDAQ NOV NTAP NFLX NWL NEM NWSA NWS NEE NLSN NKE NI NBL JWN NSC NTRS NOC NCLH NRG NUE NVDA NVR ORLY OXY OMC OKE ORCL PCAR PKG PH PAYX PYPL PNR PBCT PEP PKI PRGO PFE PM PSX PNW PXD PNC PPG PPL PFG PG PGR PLD PRU PEG PSA PHM PVH QRVO PWR QCOM DGX RL RJF RTN O REG REGN RF RSG RMD RHI ROK ROL ROP ROST RCL CRM SBAC SLB STX SEE SRE SHW SPG SWKS SLG SNA SO LUV SPGI SWK SBUX STT SYK STI SIVB SYMC SYF SNPS SYY TMUS TROW TTWO TPR TGT TEL FTI TFX TXN TXT TMO TIF TWTR TJX TSCO TDG TRV TRIP TSN UDR ULTA USB UAA UA UNP UAL UNH UPS URI UTX UHS UNM VFC VLO VAR VTR VRSN VRSK VZ VRTX VIAB V VNO VMC WAB WMT WBA DIS WM WAT WEC WCG WFC WELL WDC WU WRK WY WHR WMB WLTW WYNN XEL XRX XLNX XYL YUM ZBH ZION ZTS".split(" "))
num_tickers = len(tickers)
num_tickers

505

In [4]:
aapl = tickers.index('AAPL'); aapl

3

In [5]:
from datetime import datetime
from dateutil.parser import parse
from datetime import timedelta as Δ

time = datetime.now
now = time()
today = now.date()
today

datetime.date(2019, 11, 2)

In [6]:
data_dir = Path.cwd() / 'data'
data_dir.mkdir(parents=True, exist_ok=True)

In [7]:
from sys import executable as python
!{python} -m pip install -Uq requests
from requests import get as GET

In [8]:
import json

def fetch(date_str, ticker):
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if out_path.exists():
        return True

    print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    with out_path.open('wb') as f:
        f.write(resp.content)

    data = json.loads(resp.content)
    if data:
        return True

    return False

In [9]:
%%time
from concurrent.futures import ThreadPoolExecutor

end_date = today
start_date = datetime(2019, 8, 1).date()
N = 32

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

dates = list(get_dates(start_date, end_date))

for date in dates:
    date_str = date.strftime('%Y%m%d')
    
    with ThreadPoolExecutor(max_workers = N) as p:
        results = p.map(lambda ticker: fetch(date_str, ticker), tickers)
    
    found_data = True in results
    if not found_data:
        print('No data found for %s; breaking' % date)

Fetching data for A from 20191030Fetching data for AAL from 20191030

Fetching data for AAP from 20191030Fetching data for AAPL from 20191030
Fetching data for ABBV from 20191030

Fetching data for ABC from 20191030Fetching data for ABMD from 20191030
Fetching data for ABT from 20191030

Fetching data for ACN from 20191030
Fetching data for ADBE from 20191030
Fetching data for ADI from 20191030
Fetching data for ADM from 20191030
Fetching data for ADP from 20191030
Fetching data for ADS from 20191030
Fetching data for ADSK from 20191030
Fetching data for AEE from 20191030
Fetching data for AEP from 20191030
Fetching data for AES from 20191030
Fetching data for AFL from 20191030
Fetching data for AGN from 20191030
Fetching data for AIG from 20191030
Fetching data for AIV from 20191030
Fetching data for AIZ from 20191030
Fetching data for AJG from 20191030
Fetching data for AKAM from 20191030
Fetching data for ALB from 20191030
Fetching data for ALGN from 20191030
Fetching data for ALK f

Fetching data for HOG from 20191030
Fetching data for HOLX from 20191030
Fetching data for HON from 20191030
Fetching data for HP from 20191030
Fetching data for HPE from 20191030
Fetching data for HPQ from 20191030
Fetching data for HRB from 20191030
Fetching data for HRL from 20191030
Fetching data for HSIC from 20191030Fetching data for HST from 20191030
Fetching data for HSY from 20191030

Fetching data for HUM from 20191030
Fetching data for IBM from 20191030
Fetching data for ICE from 20191030
Fetching data for IDXX from 20191030Fetching data for IEX from 20191030
Fetching data for IFF from 20191030

Fetching data for INCY from 20191030
Fetching data for ILMN from 20191030
Fetching data for INFO from 20191030Fetching data for INTC from 20191030

Fetching data for INTU from 20191030
Fetching data for IP from 20191030
Fetching data for IPG from 20191030
Fetching data for IPGP from 20191030
Fetching data for IQV from 20191030
Fetching data for IR from 20191030
Fetching data for IRM 

Fetching data for UNM from 20191030
Fetching data for UNP from 20191030
Fetching data for UPS from 20191030
Fetching data for URI from 20191030
Fetching data for USB from 20191030
Fetching data for UTX from 20191030
Fetching data for V from 20191030
Fetching data for VAR from 20191030
Fetching data for VFC from 20191030
Fetching data for VIAB from 20191030
Fetching data for VLO from 20191030
Fetching data for VMC from 20191030
Fetching data for VNO from 20191030
Fetching data for VRSK from 20191030
Fetching data for VRSN from 20191030
Fetching data for VRTX from 20191030
Fetching data for VTR from 20191030
Fetching data for VZ from 20191030
Fetching data for WAB from 20191030
Fetching data for WAT from 20191030
Fetching data for WBA from 20191030
Fetching data for WCG from 20191030
Fetching data for WDC from 20191030
Fetching data for WEC from 20191030
Fetching data for WELL from 20191030
Fetching data for WFC from 20191030
Fetching data for WHR from 20191030
Fetching data for WLTW fro

Fetching data for FBHS from 20191031
Fetching data for FCX from 20191031
Fetching data for FDX from 20191031
Fetching data for FE from 20191031
Fetching data for FFIV from 20191031
Fetching data for FIS from 20191031
Fetching data for FISV from 20191031
Fetching data for FITB from 20191031
Fetching data for FLIR from 20191031
Fetching data for FLS from 20191031
Fetching data for FLT from 20191031
Fetching data for FMC from 20191031
Fetching data for FOX from 20191031
Fetching data for FOXA from 20191031
Fetching data for FRC from 20191031
Fetching data for FRT from 20191031Fetching data for FTI from 20191031

Fetching data for FTNT from 20191031
Fetching data for FTV from 20191031
Fetching data for GD from 20191031
Fetching data for GE from 20191031
Fetching data for GILD from 20191031
Fetching data for GIS from 20191031
Fetching data for GL from 20191031
Fetching data for GLW from 20191031
Fetching data for GM from 20191031
Fetching data for GOOG from 20191031
Fetching data for GOOGL 

Fetching data for SRE from 20191031
Fetching data for STT from 20191031
Fetching data for STX from 20191031
Fetching data for STI from 20191031
Fetching data for STZ from 20191031Fetching data for SWK from 20191031

Fetching data for SWKS from 20191031
Fetching data for SYF from 20191031
Fetching data for SYK from 20191031
Fetching data for SYMC from 20191031
Fetching data for SYY from 20191031
Fetching data for T from 20191031
Fetching data for TAP from 20191031
Fetching data for TDG from 20191031Fetching data for TEL from 20191031

Fetching data for TFX from 20191031
Fetching data for TGT from 20191031Fetching data for TIF from 20191031

Fetching data for TJX from 20191031
Fetching data for TMUS from 20191031
Fetching data for TMO from 20191031
Fetching data for TPR from 20191031
Fetching data for TRIP from 20191031Fetching data for TROW from 20191031
Fetching data for TRV from 20191031

Fetching data for TSCO from 20191031Fetching data for TSN from 20191031Fetching data for TTWO fro

Fetching data for DVA from 20191101Fetching data for DVN from 20191101

Fetching data for DXC from 20191101
Fetching data for EA from 20191101
Fetching data for EBAY from 20191101
Fetching data for ECL from 20191101
Fetching data for ED from 20191101
Fetching data for EFX from 20191101
Fetching data for EIX from 20191101
Fetching data for EL from 20191101
Fetching data for EMN from 20191101
Fetching data for EMR from 20191101Fetching data for EOG from 20191101
Fetching data for EQIX from 20191101

Fetching data for EQR from 20191101Fetching data for ES from 20191101

Fetching data for ESS from 20191101
Fetching data for ETFC from 20191101
Fetching data for ETN from 20191101
Fetching data for ETR from 20191101
Fetching data for EVRG from 20191101
Fetching data for EW from 20191101
Fetching data for EXPD from 20191101
Fetching data for EXC from 20191101
Fetching data for EXPE from 20191101Fetching data for EXR from 20191101

Fetching data for F from 20191101
Fetching data for FANG from 2

Fetching data for PXD from 20191101Fetching data for QCOM from 20191101
Fetching data for QRVO from 20191101
Fetching data for RCL from 20191101Fetching data for RE from 20191101
Fetching data for PYPL from 20191101
Fetching data for REG from 20191101


Fetching data for REGN from 20191101
Fetching data for RF from 20191101
Fetching data for RHI from 20191101
Fetching data for RJF from 20191101
Fetching data for RL from 20191101
Fetching data for RMD from 20191101
Fetching data for ROK from 20191101
Fetching data for ROL from 20191101
Fetching data for ROP from 20191101
Fetching data for ROST from 20191101
Fetching data for RSG from 20191101
Fetching data for RTN from 20191101
Fetching data for SBAC from 20191101
Fetching data for SBUX from 20191101
Fetching data for SCHW from 20191101
Fetching data for SEE from 20191101
Fetching data for SHW from 20191101
Fetching data for SIVB from 20191101
Fetching data for SJM from 20191101
Fetching data for SLB from 20191101Fetching data for SLG f

In [10]:
!{python} -m pip install -Uq pandas
from pandas import DataFrame as DF, read_csv, read_json
import pandas as pd

In [11]:
minutes = 390  # [9:30am,4:00pm)

In [12]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]
num_features = len(features)
cols = [ 'datetime', 'ticker' ] + features

In [20]:
features = [ 'average' ]
num_features = len(features)

In [21]:
def load_data_arr(date, ticker):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if not out_path.exists():
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    df = read_json(out_path)
    if df.empty:
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    arr = df[features].values
    assert arr.shape == (minutes, len(features))
    return arr

In [35]:
!{python} -m pip install -Uq numpy
import numpy as np
from numpy import array, nan, zeros, count_nonzero as cnz, isnan as na, mean, std
from numpy.random import shuffle, permutation

In [ ]:
def load_date_arr(date):
    arr = array([ 
        load_data_arr(start_date, ticker) 
        for ticker in tickers 
    ]) \
    .reshape((
        minutes, 
        len(tickers), 
        len(features),
    ))
    assert arr.shape == (minutes, num_tickers, len(features))
    return arr

In [15]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

In [16]:
ticker = 'AAPL'
num_tickers = 1

In [58]:
aapl = array([ load_data_arr(date, ticker) for date in dates ]); aapl.shape

(67, 390, 1)

In [59]:
shape = aapl.shape
aapl = aapl.reshape((shape[0] * shape[1], shape[2]))
avgs = aapl[:, features.index('average')]
shape = aapl.shape; shape

(26130, 1)

In [60]:
y = np.roll(avgs, -1)
y = y[:-1]
x = aapl[:-1]
x.shape, y.shape

((26129, 1), (26129,))

In [61]:
window = 1
n = x.shape[0]
x = np.array([ x[i:(i+window)] for i in range(n-window+1) ])
y = y[(window-1):]
x.shape, y.shape

((26129, 1, 1), (26129,))

In [62]:
idxs = np.logical_and([ (cnz(na(row)) == 0) for row in x ], ~na(y))
y = y[idxs]
x = x[idxs]
x.shape, y.shape

((24432, 1, 1), (24432,))

In [63]:
n = x.shape[0]

In [64]:
mean((x[:, -1, 0] - y)**2)

0.025807457924034084

In [65]:
ox = x.copy()
oy = y.copy()

In [66]:
perm = permutation(n); perm

array([22949, 10771, 13722, ..., 13469, 12427, 19514])

In [67]:
x = x[perm]
y = y[perm]

In [68]:
val_split = 0.1
tn = int((1 - val_split) * n)
vn = n - tn
tx = x[:tn]
ty = y[:tn]
vx = x[tn:]
vy = y[tn:]
[ a.shape for a in [tx,ty,vx,vy] ]

[(21988, 1, 1), (21988,), (2444, 1, 1), (2444,)]

In [69]:
lasts = tx[:, -1, 0]

In [74]:
mean(tx), std(tx), mean(ty), std(ty)

(-4.575807321431244e-16, 1.0, 0.00012986712011287224, 1.0001142196576933)

In [71]:
u = mean(lasts)
s = std(lasts)
x = (x - u) / s
y = (y - u) / s
mean(x), std(x), mean(y), std(y)

(-0.001778588936448751,
 1.0000236994621259,
 -0.001649276858867785,
 1.0001426984102393)

In [72]:
val_split = 0.1
tn = int((1 - val_split) * n)
vn = n - tn
tx = x[:tn]
ty = y[:tn]
vx = x[tn:]
vy = y[tn:]
[ a.shape for a in [tx,ty,vx,vy] ]

[(21988, 1, 1), (21988,), (2444, 1, 1), (2444,)]

In [76]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras import Sequential

In [78]:
model = Sequential()
model.add(Dense(4, input_shape=(1, 1)))
model.add(Dense(1))
model.build()
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1, 4)              8         
_________________________________________________________________
dense_3 (Dense)              (None, 1, 1)              5         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=50,
          epochs=10)

px = model.predict(vx)
px.max() - px.min(), x[:10, -1, 0], y[:10], px[:10]

Train on 21988 samples, validate on 2444 samples
Epoch 1/10
21988/21988 [==============================] - 1s 57us/sample - loss: 1.1273 - val_loss: 0.0980
Epoch 2/10
21988/21988 [==============================] - 1s 23us/sample - loss: 0.0149 - val_loss: 1.5000e-04
Epoch 3/10
21988/21988 [==============================] - 0s 22us/sample - loss: 1.2314e-04 - val_loss: 1.3653e-04
Epoch 4/10
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2213e-04 - val_loss: 1.3597e-04
Epoch 5/10
21988/21988 [==============================] - 1s 25us/sample - loss: 1.2214e-04 - val_loss: 1.3604e-04
Epoch 6/10
21988/21988 [==============================] - 0s 21us/sample - loss: 1.2213e-04 - val_loss: 1.3614e-04
Epoch 7/10
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2229e-04 - val_loss: 1.3608e-04
Epoch 8/10
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2232e-04 - val_loss: 1.3658e-04
Epoch 9/10
21988/21988 [===================

(4.336399, array([[[ 2.04760044]],
 
        [[ 0.04591645]],
 
        [[-0.07633646]],
 
        [[ 0.26960885]],
 
        [[-0.95298941]],
 
        [[ 0.05366098]],
 
        [[-1.0759338 ]],
 
        [[-1.76450535]],
 
        [[-0.43230818]],
 
        [[-1.61044455]]]), array([ 2.04787703,  0.04605474, -0.08975109,  0.28094905, -0.95222879,
         0.05538967, -1.07399767, -1.7786806 , -0.43265391, -1.60152451]), array([[[-1.0935761 ]],
 
        [[-1.1460584 ]],
 
        [[-1.098209  ]],
 
        [[-0.59475166]],
 
        [[ 1.1017634 ]],
 
        [[ 0.389552  ]],
 
        [[ 0.03303147]],
 
        [[-0.54800844]],
 
        [[ 0.15763387]],
 
        [[-0.5451043 ]]], dtype=float32))

In [80]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=50,
          epochs=10)

px = model.predict(x)
px.max() - px.min(), x[:10, -1, 0], y[:10], px[:10]

Train on 21988 samples, validate on 2444 samples
Epoch 1/10
21988/21988 [==============================] - 0s 22us/sample - loss: 1.2252e-04 - val_loss: 1.3634e-04
Epoch 2/10
21988/21988 [==============================] - 1s 26us/sample - loss: 1.2263e-04 - val_loss: 1.3600e-04
Epoch 3/10
21988/21988 [==============================] - 1s 27us/sample - loss: 1.2269e-04 - val_loss: 1.3621e-04
Epoch 4/10
21988/21988 [==============================] - 1s 25us/sample - loss: 1.2306e-04 - val_loss: 1.3710e-04
Epoch 5/10
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2308e-04 - val_loss: 1.3594e-04
Epoch 6/10
21988/21988 [==============================] - 1s 27us/sample - loss: 1.2359e-04 - val_loss: 1.3630e-04
Epoch 7/10
21988/21988 [==============================] - 1s 28us/sample - loss: 1.2366e-04 - val_loss: 1.3662e-04
Epoch 8/10
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2377e-04 - val_loss: 1.3702e-04
Epoch 9/10
21988/21988 [=======

(4.3377385,
 array([ 2.04760044,  0.04591645, -0.07633646,  0.26960885, -0.95298941,
         0.05366098, -1.0759338 , -1.76450535, -0.43230818, -1.61044455]),
 array([ 2.04787703,  0.04605474, -0.08975109,  0.28094905, -0.95222879,
         0.05538967, -1.07399767, -1.7786806 , -0.43265391, -1.60152451]),
 array([[[-1.092921  ]],
 
        [[-1.1454197 ]],
 
        [[-1.0975553 ]],
 
        [[-0.59394234]],
 
        [[ 1.1030966 ]],
 
        [[ 0.39066538]],
 
        [[ 0.03403469]],
 
        [[-0.5471847 ]],
 
        [[ 0.15867561]],
 
        [[-0.5442797 ]]], dtype=float32))

In [81]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=50,
          epochs=10)

px = model.predict(x)
px.max() - px.min(), x[:10, -1, 0], y[:10], px[:10]

Train on 21988 samples, validate on 2444 samples
Epoch 1/10
21988/21988 [==============================] - 0s 21us/sample - loss: 1.2453e-04 - val_loss: 1.3741e-04
Epoch 2/10
21988/21988 [==============================] - 0s 23us/sample - loss: 1.2571e-04 - val_loss: 1.3935e-04
Epoch 3/10
21988/21988 [==============================] - 0s 21us/sample - loss: 1.2562e-04 - val_loss: 1.4338e-04
Epoch 4/10
21988/21988 [==============================] - 0s 21us/sample - loss: 1.2582e-04 - val_loss: 1.3629e-04
Epoch 5/10
21988/21988 [==============================] - 0s 22us/sample - loss: 1.2575e-04 - val_loss: 1.3788e-04
Epoch 6/10
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2638e-04 - val_loss: 1.3700e-04
Epoch 7/10
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2744e-04 - val_loss: 1.3875e-04
Epoch 8/10
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2620e-04 - val_loss: 1.3717e-04
Epoch 9/10
21988/21988 [=======

(4.354834,
 array([ 2.04760044,  0.04591645, -0.07633646,  0.26960885, -0.95298941,
         0.05366098, -1.0759338 , -1.76450535, -0.43230818, -1.61044455]),
 array([ 2.04787703,  0.04605474, -0.08975109,  0.28094905, -0.95222879,
         0.05538967, -1.07399767, -1.7786806 , -0.43265391, -1.60152451]),
 array([[[ 2.0452137 ]],
 
        [[ 0.04648627]],
 
        [[-0.07558606]],
 
        [[ 0.2698483 ]],
 
        [[-0.95094407]],
 
        [[ 0.05421936]],
 
        [[-1.0737069 ]],
 
        [[-1.7612613 ]],
 
        [[-0.43103197]],
 
        [[-1.6074281 ]]], dtype=float32))

In [86]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=50,
          epochs=20)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/20
21988/21988 [==============================] - 0s 21us/sample - loss: 1.2821e-04 - val_loss: 1.3688e-04
Epoch 2/20
21988/21988 [==============================] - 0s 21us/sample - loss: 1.2678e-04 - val_loss: 1.3689e-04
Epoch 3/20
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2662e-04 - val_loss: 1.5382e-04
Epoch 4/20
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2766e-04 - val_loss: 1.3838e-04
Epoch 5/20
21988/21988 [==============================] - 0s 22us/sample - loss: 1.2615e-04 - val_loss: 1.4403e-04
Epoch 6/20
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2677e-04 - val_loss: 1.4841e-04
Epoch 7/20
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2867e-04 - val_loss: 1.3683e-04
Epoch 8/20
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2633e-04 - val_loss: 1.3685e-04
Epoch 9/20
21988/21988 [=======

4.349675

In [98]:
w = model.get_weights(); w, [ l.shape for l in w ]

([array([[-1.0088345 ,  0.5443364 , -0.4205381 ,  0.49378547]],
        dtype=float32),
  array([ 8.3063787e-05, -5.9511018e-04, -1.6151814e-03,  2.1109341e-02],
        dtype=float32),
  array([[-0.8659785 ],
         [ 0.7806577 ],
         [ 0.7433081 ],
         [ 0.02300616]], dtype=float32),
  array([-0.00075723], dtype=float32)],
 [(1, 4), (4,), (4, 1), (1,)])

In [83]:
mean((tx[:, -1, 0]-ty)**2)

0.00012199883962722794

In [99]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=50,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 1s 27us/sample - loss: 1.2702e-04 - val_loss: 1.3745e-04
Epoch 2/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2675e-04 - val_loss: 1.3725e-04
Epoch 3/100
21988/21988 [==============================] - 1s 26us/sample - loss: 1.2766e-04 - val_loss: 1.3874e-04
Epoch 4/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2651e-04 - val_loss: 1.4087e-04
Epoch 5/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2740e-04 - val_loss: 1.4409e-04
Epoch 6/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2663e-04 - val_loss: 1.4659e-04
Epoch 7/100
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2643e-04 - val_loss: 1.3708e-04
Epoch 8/100
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2760e-04 - val_loss: 1.3663e-04
Epoch 9/100
21988/21988

21988/21988 [==============================] - 1s 23us/sample - loss: 1.2641e-04 - val_loss: 1.4072e-04
Epoch 71/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2622e-04 - val_loss: 1.4017e-04
Epoch 72/100
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2643e-04 - val_loss: 1.4725e-04
Epoch 73/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2658e-04 - val_loss: 1.3806e-04
Epoch 74/100
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2681e-04 - val_loss: 1.3913e-04
Epoch 75/100
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2740e-04 - val_loss: 1.4045e-04
Epoch 76/100
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2666e-04 - val_loss: 1.3753e-04
Epoch 77/100
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2715e-04 - val_loss: 1.3612e-04
Epoch 78/100
21988/21988 [==============================] - 1s 31us/sample - 

(4.360306, array([[ 2.04760044,  2.04787703,  2.04826546],
        [ 0.04591645,  0.04605474,  0.04702692],
        [-0.07633646, -0.08975109, -0.07519878],
        [ 0.26960885,  0.28094905,  0.27066955],
        [-0.95298941, -0.95222879, -0.95165652],
        [ 0.05366098,  0.05538967,  0.05476972],
        [-1.0759338 , -1.07399767, -1.07457376],
        [-1.76450535, -1.7786806 , -1.76299202],
        [-0.43230818, -0.43265391, -0.43109128],
        [-1.61044455, -1.60152451, -1.60896575]]))

In [100]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=50,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2678e-04 - val_loss: 1.3734e-04
Epoch 2/100
21988/21988 [==============================] - 1s 26us/sample - loss: 1.2582e-04 - val_loss: 1.3799e-04A: 0s - loss: 1.2981e
Epoch 3/100
21988/21988 [==============================] - 1s 28us/sample - loss: 1.2584e-04 - val_loss: 1.3637e-04
Epoch 4/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2701e-04 - val_loss: 1.4162e-04
Epoch 5/100
21988/21988 [==============================] - 0s 22us/sample - loss: 1.2654e-04 - val_loss: 1.4443e-04
Epoch 6/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2632e-04 - val_loss: 1.3888e-04
Epoch 7/100
21988/21988 [==============================] - 1s 24us/sample - loss: 1.2588e-04 - val_loss: 1.3952e-04
Epoch 8/100
21988/21988 [==============================] - 0s 22us/sample - loss: 1.2662e-04 - val_loss: 1.3753e-04
Ep

21988/21988 [==============================] - 0s 20us/sample - loss: 1.2775e-04 - val_loss: 1.3615e-04
Epoch 71/100
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2554e-04 - val_loss: 1.3673e-04
Epoch 72/100
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2680e-04 - val_loss: 1.3786e-04
Epoch 73/100
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2682e-04 - val_loss: 1.6533e-04
Epoch 74/100
21988/21988 [==============================] - 0s 20us/sample - loss: 1.2693e-04 - val_loss: 1.4108e-04
Epoch 75/100
21988/21988 [==============================] - 0s 22us/sample - loss: 1.2580e-04 - val_loss: 1.3919e-04
Epoch 76/100
21988/21988 [==============================] - 1s 26us/sample - loss: 1.2692e-04 - val_loss: 1.4013e-04
Epoch 77/100
21988/21988 [==============================] - 1s 23us/sample - loss: 1.2666e-04 - val_loss: 1.3867e-04
Epoch 78/100
21988/21988 [==============================] - 1s 23us/sample - 

(4.3640604, array([[ 2.04760044,  2.04787703,  2.04689789],
        [ 0.04591645,  0.04605474,  0.04393584],
        [-0.07633646, -0.08975109, -0.07839511],
        [ 0.26960885,  0.28094905,  0.26777107],
        [-0.95298941, -0.95222879, -0.95560771],
        [ 0.05366098,  0.05538967,  0.05168532],
        [-1.0759338 , -1.07399767, -1.07863069],
        [-1.76450535, -1.7786806 , -1.76764166],
        [-0.43230818, -0.43265391, -0.43459409],
        [-1.61044455, -1.60152451, -1.61348259]]))

In [101]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=1000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2333e-04 - val_loss: 1.3600e-04
Epoch 2/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2216e-04 - val_loss: 1.3636e-04
Epoch 3/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2211e-04 - val_loss: 1.3595e-04
Epoch 4/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2219e-04 - val_loss: 1.3616e-04
Epoch 5/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2249e-04 - val_loss: 1.3601e-04
Epoch 6/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2220e-04 - val_loss: 1.3598e-04
Epoch 7/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2219e-04 - val_loss: 1.3654e-04
Epoch 8/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2217e-04 - val_loss: 1.3596e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 2us/sample - loss: 1.2228e-04 - val_loss: 1.3666e-04
Epoch 72/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2246e-04 - val_loss: 1.3623e-04
Epoch 73/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2228e-04 - val_loss: 1.3659e-04
Epoch 74/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2237e-04 - val_loss: 1.3600e-04
Epoch 75/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2239e-04 - val_loss: 1.3595e-04
Epoch 76/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2224e-04 - val_loss: 1.3642e-04
Epoch 77/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2227e-04 - val_loss: 1.3664e-04
Epoch 78/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2255e-04 - val_loss: 1.3661e-04
Epoch 79/100
21988/21988 [==============================] - 0s 2us/sample - loss: 1.2

(4.359906, array([[ 2.04760044,  2.04787703,  2.04632545],
        [ 0.04591645,  0.04605474,  0.04527004],
        [-0.07633646, -0.08975109, -0.07694449],
        [ 0.26960885,  0.28094905,  0.2688922 ],
        [-0.95298941, -0.95222879, -0.95332211],
        [ 0.05366098,  0.05538967,  0.05301214],
        [-1.0759338 , -1.07399767, -1.0762279 ],
        [-1.76450535, -1.7786806 , -1.76458323],
        [-0.43230818, -0.43265391, -0.43280444],
        [-1.61044455, -1.60152451, -1.61057079]]))

In [102]:
w = model.get_weights(); w, [ l.shape for l in w ]

([array([[-1.0015891e+00,  5.3329176e-01, -3.6671025e-01,  3.9901586e-16]],
        dtype=float32),
  array([-4.536753e-03,  4.788202e-03, -9.803617e-03,  6.582850e-17],
        dtype=float32),
  array([[-8.393317e-01],
         [ 7.140207e-01],
         [ 6.047303e-01],
         [-1.225996e-17]], dtype=float32),
  array([-0.00193016], dtype=float32)],
 [(1, 4), (4,), (4, 1), (1,)])

In [104]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2282e-04 - val_loss: 1.3605e-04
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2217e-04 - val_loss: 1.3626e-04
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2217e-04 - val_loss: 1.3617e-04
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2227e-04 - val_loss: 1.3597e-04
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3605e-04
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2210e-04 - val_loss: 1.3606e-04
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2207e-04 - val_loss: 1.3598e-04
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2218e-04 - val_loss: 1.3596e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 1us/sample - loss: 1.2212e-04 - val_loss: 1.3598e-04
Epoch 72/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2203e-04 - val_loss: 1.3609e-04
Epoch 73/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2209e-04 - val_loss: 1.3596e-04
Epoch 74/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3595e-04
Epoch 75/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3602e-04
Epoch 76/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3600e-04
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2206e-04 - val_loss: 1.3595e-04
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2202e-04 - val_loss: 1.3595e-04
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2

(4.361722, array([[ 2.04760044,  2.04787703,  2.04798245],
        [ 0.04591645,  0.04605474,  0.04609357],
        [-0.07633646, -0.08975109, -0.07617185],
        [ 0.26960885,  0.28094905,  0.26980886],
        [-0.95298941, -0.95222879, -0.95291436],
        [ 0.05366098,  0.05538967,  0.05383889],
        [-1.0759338 , -1.07399767, -1.07587147],
        [-1.76450535, -1.7786806 , -1.76451361],
        [-0.43230818, -0.43265391, -0.43217996],
        [-1.61044455, -1.60152451, -1.61043692]]))

In [105]:
w = model.get_weights(); w, [ l.shape for l in w ]

([array([[-1.0016874e+00,  5.3338945e-01, -3.6659673e-01,  1.2688262e-16]],
        dtype=float32),
  array([-4.7905757e-03,  5.0417539e-03, -9.5493123e-03, -9.6088791e-18],
        dtype=float32),
  array([[-8.3942842e-01],
         [ 7.1410954e-01],
         [ 6.0459095e-01],
         [ 1.1195497e-19]], dtype=float32),
  array([-0.00167586], dtype=float32)],
 [(1, 4), (4,), (4, 1), (1,)])

In [106]:
mean((x[:, -1, 0] - y)**2)

0.00012339544539288403

In [107]:
mean((tx[:, -1, 0] - ty)**2)

0.00012199883962722794

In [121]:
model.fit(x, y, 
          batch_size=n,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

Train on 24432 samples
Epoch 1/100
24432/24432 [==============================] - 0s 16us/sample - loss: 1.2338e-04
Epoch 2/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2346e-04
Epoch 3/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2352e-04
Epoch 4/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2345e-04
Epoch 5/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 6/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2341e-04
Epoch 7/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2347e-04
Epoch 8/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2344e-04
Epoch 9/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2339e-04
Epoch 10/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2339e-04
Epoch 11/100
24432/24432 [=============================

Epoch 89/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 90/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 91/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 92/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 93/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 94/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 95/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 96/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 97/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 98/100
24432/24432 [==============================] - 0s 0us/sample - loss: 1.2338e-04
Epoch 99/100
24432/24432 [==============================] - 0s 0us/sam

(4.3615174, array([[ 2.04760044,  2.04787703,  2.04784369],
        [ 0.04591645,  0.04605474,  0.04604882],
        [-0.07633646, -0.08975109, -0.07621087],
        [ 0.26960885,  0.28094905,  0.26975361],
        [-0.95298941, -0.95222879, -0.95291239],
        [ 0.05366098,  0.05538967,  0.05379378],
        [-1.0759338 , -1.07399767, -1.07586348],
        [-1.76450535, -1.7786806 , -1.76447332],
        [-0.43230818, -0.43265391, -0.43220228],
        [-1.61044455, -1.60152451, -1.61040401]]))

In [122]:
w = model.get_weights(); w, [ l.shape for l in w ]

([array([[-1.0016755e+00,  5.3337759e-01, -3.6660731e-01,  1.1738654e-16]],
        dtype=float32),
  array([-4.7772164e-03,  5.0284215e-03, -9.5628640e-03, -6.3762692e-18],
        dtype=float32),
  array([[-8.3941674e-01],
         [ 7.1409714e-01],
         [ 6.0460109e-01],
         [-2.0790356e-19]], dtype=float32),
  array([-0.00168932], dtype=float32)],
 [(1, 4), (4,), (4, 1), (1,)])

In [109]:
m = Sequential()
m.add(Dense(4, input_shape=(1, 1)))
m.add(Dense(1))
m.build()
m.compile(loss='mse', optimizer='adam')
m.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1, 4)              8         
_________________________________________________________________
dense_7 (Dense)              (None, 1, 1)              5         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [114]:
m.set_weights(array([
    array([1, 0, 0, 0]).reshape((1, 4)),
    array([0, 0, 0, 0]),
    array([1, 0, 0, 0]).reshape((4, 1)),
    array([0])
]))

In [115]:
m.get_weights()

[array([[1., 0., 0., 0.]], dtype=float32),
 array([0., 0., 0., 0.], dtype=float32),
 array([[1.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 array([0.], dtype=float32)]

In [116]:
px = m.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

(4.3612757, array([[ 2.04760044,  2.04787703,  2.04760051],
        [ 0.04591645,  0.04605474,  0.04591645],
        [-0.07633646, -0.08975109, -0.07633647],
        [ 0.26960885,  0.28094905,  0.26960886],
        [-0.95298941, -0.95222879, -0.9529894 ],
        [ 0.05366098,  0.05538967,  0.05366098],
        [-1.0759338 , -1.07399767, -1.07593381],
        [-1.76450535, -1.7786806 , -1.76450539],
        [-0.43230818, -0.43265391, -0.43230817],
        [-1.61044455, -1.60152451, -1.61044455]]))

In [117]:
m.evaluate(tx, ty)

21988/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.00012199884220183997

In [120]:
mean((tx[:, -1, 0]-ty)**2)

0.00012199883962722794

In [118]:
m.evaluate(vx, vy)

2444/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.00013596029341178558

In [119]:
mean((vx[:, -1, 0]-vy)**2)

0.00013596032574282095

In [104]:
m.fit(x, y, 
      validation_split=0.1,
      batch_size=10000,
      epochs=100)

px = m.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2282e-04 - val_loss: 1.3605e-04
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2217e-04 - val_loss: 1.3626e-04
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2217e-04 - val_loss: 1.3617e-04
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2227e-04 - val_loss: 1.3597e-04
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3605e-04
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2210e-04 - val_loss: 1.3606e-04
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2207e-04 - val_loss: 1.3598e-04
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2218e-04 - val_loss: 1.3596e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 1us/sample - loss: 1.2212e-04 - val_loss: 1.3598e-04
Epoch 72/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2203e-04 - val_loss: 1.3609e-04
Epoch 73/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2209e-04 - val_loss: 1.3596e-04
Epoch 74/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3595e-04
Epoch 75/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3602e-04
Epoch 76/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2204e-04 - val_loss: 1.3600e-04
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2206e-04 - val_loss: 1.3595e-04
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2202e-04 - val_loss: 1.3595e-04
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2

(4.361722, array([[ 2.04760044,  2.04787703,  2.04798245],
        [ 0.04591645,  0.04605474,  0.04609357],
        [-0.07633646, -0.08975109, -0.07617185],
        [ 0.26960885,  0.28094905,  0.26980886],
        [-0.95298941, -0.95222879, -0.95291436],
        [ 0.05366098,  0.05538967,  0.05383889],
        [-1.0759338 , -1.07399767, -1.07587147],
        [-1.76450535, -1.7786806 , -1.76451361],
        [-0.43230818, -0.43265391, -0.43217996],
        [-1.61044455, -1.60152451, -1.61043692]]))

In [123]:
model = Sequential()
model.add(SimpleRNN(1, input_shape=(1, 1)))
model.add(Dense(1))
model.build()
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1)                 3         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [126]:
px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

(0.42466536, array([[ 2.04760044,  2.04787703, -0.21233439],
        [ 0.04591645,  0.04605474, -0.01577247],
        [-0.07633646, -0.08975109,  0.02613747],
        [ 0.26960885,  0.28094905, -0.08732236],
        [-0.95298941, -0.95222879,  0.1942067 ],
        [ 0.05366098,  0.05538967, -0.01842041],
        [-1.0759338 , -1.07399767,  0.2001546 ],
        [-1.76450535, -1.7786806 ,  0.2114878 ],
        [-0.43230818, -0.43265391,  0.12850632],
        [-1.61044455, -1.60152451,  0.2105801 ]]))

In [127]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 1s 44us/sample - loss: 1.3357 - val_loss: 1.3314
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.3303 - val_loss: 1.3260
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.3251 - val_loss: 1.3207
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.3198 - val_loss: 1.3155
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.3146 - val_loss: 1.3102
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.3094 - val_loss: 1.3050
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.3042 - val_loss: 1.2998
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2990 - val_loss: 1.2947
Epoch 9/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.2938 - val_

21988/21988 [==============================] - 0s 1us/sample - loss: 0.9951 - val_loss: 0.9927
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9913 - val_loss: 0.9888
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9874 - val_loss: 0.9850
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9836 - val_loss: 0.9812
Epoch 80/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9798 - val_loss: 0.9774
Epoch 81/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9760 - val_loss: 0.9736
Epoch 82/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9722 - val_loss: 0.9698
Epoch 83/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9684 - val_loss: 0.9661
Epoch 84/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9646 - val_loss: 0.9623
Epoch 85/100
21988/21988 [===============

(0.1413452, array([[ 2.04760044,  2.04787703,  0.07080309],
        [ 0.04591645,  0.04605474,  0.01503733],
        [-0.07633646, -0.08975109,  0.0023541 ],
        [ 0.26960885,  0.28094905,  0.03542323],
        [-0.95298941, -0.95222879, -0.06069516],
        [ 0.05366098,  0.05538967,  0.01581674],
        [-1.0759338 , -1.07399767, -0.06371349],
        [-1.76450535, -1.7786806 , -0.07009631],
        [-0.43230818, -0.43265391, -0.03262189],
        [-1.61044455, -1.60152451, -0.06952286]]))

In [128]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.9015 - val_loss: 0.8994
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8978 - val_loss: 0.8957
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8942 - val_loss: 0.8921
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8906 - val_loss: 0.8884
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8869 - val_loss: 0.8848
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8833 - val_loss: 0.8812
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8797 - val_loss: 0.8776
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8760 - val_loss: 0.8740
Epoch 9/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.8724 - val_l

21988/21988 [==============================] - 0s 1us/sample - loss: 0.6466 - val_loss: 0.6460
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6436 - val_loss: 0.6430
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6406 - val_loss: 0.6401
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6376 - val_loss: 0.6371
Epoch 80/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6346 - val_loss: 0.6341
Epoch 81/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6316 - val_loss: 0.6312
Epoch 82/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6286 - val_loss: 0.6283
Epoch 83/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6256 - val_loss: 0.6254
Epoch 84/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.6227 - val_loss: 0.6224
Epoch 85/100
21988/21988 [===============

(0.6600971, array([[ 2.04760044,  2.04787703,  0.36353934],
        [ 0.04591645,  0.04605474,  0.02291597],
        [-0.07633646, -0.08975109, -0.0482162 ],
        [ 0.26960885,  0.28094905,  0.15090995],
        [-0.95298941, -0.95222879, -0.28032681],
        [ 0.05366098,  0.05538967,  0.02752898],
        [-1.0759338 , -1.07399767, -0.28611958],
        [-1.76450535, -1.7786806 , -0.29585987],
        [-0.43230818, -0.43265391, -0.2022467 ],
        [-1.61044455, -1.60152451, -0.29519203]]))

In [129]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5743 - val_loss: 0.5744
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5715 - val_loss: 0.5716
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5688 - val_loss: 0.5689
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5661 - val_loss: 0.5662
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5634 - val_loss: 0.5635
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5607 - val_loss: 0.5609
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5580 - val_loss: 0.5582
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5554 - val_loss: 0.5556
Epoch 9/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.5527 - val_l

21988/21988 [==============================] - 0s 1us/sample - loss: 0.3998 - val_loss: 0.4014
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3979 - val_loss: 0.3995
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3960 - val_loss: 0.3976
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3941 - val_loss: 0.3957
Epoch 80/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3922 - val_loss: 0.3938
Epoch 81/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3904 - val_loss: 0.3920
Epoch 82/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3885 - val_loss: 0.3901
Epoch 83/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3867 - val_loss: 0.3883
Epoch 84/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3848 - val_loss: 0.3865
Epoch 85/100
21988/21988 [===============

(1.1117752, array([[ 2.04760044,  2.04787703,  0.62567115],
        [ 0.04591645,  0.04605474,  0.03074823],
        [-0.07633646, -0.08975109, -0.10368007],
        [ 0.26960885,  0.28094905,  0.27703625],
        [-0.95298941, -0.95222879, -0.47077867],
        [ 0.05366098,  0.05538967,  0.03960471],
        [-1.0759338 , -1.07399767, -0.47685841],
        [-1.76450535, -1.7786806 , -0.48563269],
        [-0.43230818, -0.43265391, -0.36653438],
        [-1.61044455, -1.60152451, -0.48514649]]))

In [130]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3550 - val_loss: 0.3569
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3533 - val_loss: 0.3552
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3517 - val_loss: 0.3536
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3500 - val_loss: 0.3519
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3484 - val_loss: 0.3503
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3468 - val_loss: 0.3487
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3452 - val_loss: 0.3471
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3436 - val_loss: 0.3455
Epoch 9/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.3420 - val_l

21988/21988 [==============================] - 0s 1us/sample - loss: 0.2532 - val_loss: 0.2558
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2521 - val_loss: 0.2547
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2511 - val_loss: 0.2537
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2500 - val_loss: 0.2526
Epoch 80/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2489 - val_loss: 0.2516
Epoch 81/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2479 - val_loss: 0.2506
Epoch 82/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2469 - val_loss: 0.2496
Epoch 83/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2458 - val_loss: 0.2485
Epoch 84/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2448 - val_loss: 0.2475
Epoch 85/100
21988/21988 [===============

(1.4820375, array([[ 2.04760044,  2.04787703,  0.84166276],
        [ 0.04591645,  0.04605474,  0.02800459],
        [-0.07633646, -0.08975109, -0.15735994],
        [ 0.26960885,  0.28094905,  0.37368566],
        [-0.95298941, -0.95222879, -0.62455648],
        [ 0.05366098,  0.05538967,  0.04034539],
        [-1.0759338 , -1.07399767, -0.63106543],
        [-1.76450535, -1.7786806 , -0.63993627],
        [-0.43230818, -0.43265391, -0.50161207],
        [-1.61044455, -1.60152451, -0.63948494]]))

In [131]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2283 - val_loss: 0.2311
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2274 - val_loss: 0.2302
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2265 - val_loss: 0.2293
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2256 - val_loss: 0.2284
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2247 - val_loss: 0.2275
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2238 - val_loss: 0.2266
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2229 - val_loss: 0.2258
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2220 - val_loss: 0.2249
Epoch 9/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.2212 - val_l

21988/21988 [==============================] - 0s 1us/sample - loss: 0.1730 - val_loss: 0.1761
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1724 - val_loss: 0.1755
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1718 - val_loss: 0.1749
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1713 - val_loss: 0.1743
Epoch 80/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1707 - val_loss: 0.1738
Epoch 81/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1701 - val_loss: 0.1732
Epoch 82/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1695 - val_loss: 0.1726
Epoch 83/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1690 - val_loss: 0.1721
Epoch 84/100
21988/21988 [==============================] - 0s 1us/sample - loss: 0.1684 - val_loss: 0.1715
Epoch 85/100
21988/21988 [===============

(1.7757287, array([[ 2.04760044,  2.04787703,  1.01584601],
        [ 0.04591645,  0.04605474,  0.0224751 ],
        [-0.07633646, -0.08975109, -0.18835512],
        [ 0.26960885,  0.28094905,  0.42194608],
        [-0.95298941, -0.95222879, -0.73785913],
        [ 0.05366098,  0.05538967,  0.03654341],
        [-1.0759338 , -1.07399767, -0.74659038],
        [-1.76450535, -1.7786806 , -0.7591309 ],
        [-0.43230818, -0.43265391, -0.58589429],
        [-1.61044455, -1.60152451, -0.75844032]]))

In [132]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=1000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 3us/sample - loss: 0.1577 - val_loss: 0.1592
Epoch 2/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1540 - val_loss: 0.1556
Epoch 3/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1505 - val_loss: 0.1521
Epoch 4/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1471 - val_loss: 0.1488
Epoch 5/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1438 - val_loss: 0.1455
Epoch 6/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1406 - val_loss: 0.1423
Epoch 7/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1375 - val_loss: 0.1393
Epoch 8/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1345 - val_loss: 0.1363
Epoch 9/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.1316 - val_l

21988/21988 [==============================] - 0s 2us/sample - loss: 0.0378 - val_loss: 0.0392
Epoch 77/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0373 - val_loss: 0.0386
Epoch 78/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0368 - val_loss: 0.0381
Epoch 79/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0362 - val_loss: 0.0375
Epoch 80/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0357 - val_loss: 0.0370
Epoch 81/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0352 - val_loss: 0.0365
Epoch 82/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0347 - val_loss: 0.0360
Epoch 83/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0343 - val_loss: 0.0355
Epoch 84/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0338 - val_loss: 0.0350
Epoch 85/100
21988/21988 [===============

(2.9492817, array([[ 2.04760044,  2.04787703,  1.6106168 ],
        [ 0.04591645,  0.04605474,  0.02653702],
        [-0.07633646, -0.08975109, -0.13219182],
        [ 0.26960885,  0.28094905,  0.32301018],
        [-0.95298941, -0.95222879, -0.97094506],
        [ 0.05366098,  0.05538967,  0.03673962],
        [-1.0759338 , -1.07399767, -1.03867209],
        [-1.76450535, -1.7786806 , -1.25986612],
        [-0.43230818, -0.43265391, -0.54791355],
        [-1.61044455, -1.60152451, -1.22822607]]))

In [133]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=1000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 3us/sample - loss: 0.0271 - val_loss: 0.0282
Epoch 2/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0267 - val_loss: 0.0278
Epoch 3/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0264 - val_loss: 0.0275
Epoch 4/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0261 - val_loss: 0.0271
Epoch 5/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0258 - val_loss: 0.0268
Epoch 6/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0255 - val_loss: 0.0265
Epoch 7/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0252 - val_loss: 0.0262
Epoch 8/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0249 - val_loss: 0.0259
Epoch 9/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0246 - val_l

21988/21988 [==============================] - 0s 2us/sample - loss: 0.0123 - val_loss: 0.0129
Epoch 77/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0122 - val_loss: 0.0128
Epoch 78/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0121 - val_loss: 0.0127
Epoch 79/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0120 - val_loss: 0.0126
Epoch 80/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0119 - val_loss: 0.0125
Epoch 81/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0118 - val_loss: 0.0123
Epoch 82/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0116 - val_loss: 0.0122
Epoch 83/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0115 - val_loss: 0.0121
Epoch 84/100
21988/21988 [==============================] - 0s 2us/sample - loss: 0.0114 - val_loss: 0.0120
Epoch 85/100
21988/21988 [===============

(3.4494188, array([[ 2.04760044,  2.04787703,  1.80785644],
        [ 0.04591645,  0.04605474,  0.03870088],
        [-0.07633646, -0.08975109, -0.10609728],
        [ 0.26960885,  0.28094905,  0.30666786],
        [-0.95298941, -0.95222879, -0.98703671],
        [ 0.05366098,  0.05538967,  0.04794367],
        [-1.0759338 , -1.07399767, -1.07940495],
        [-1.76450535, -1.7786806 , -1.45424604],
        [-0.43230818, -0.43265391, -0.50530148],
        [-1.61044455, -1.60152451, -1.38937724]]))

In [134]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=100,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 18us/sample - loss: 0.0095 - val_loss: 0.0096
Epoch 2/100
21988/21988 [==============================] - 0s 21us/sample - loss: 0.0087 - val_loss: 0.0089
Epoch 3/100
21988/21988 [==============================] - 0s 17us/sample - loss: 0.0081 - val_loss: 0.0082
Epoch 4/100
21988/21988 [==============================] - 0s 17us/sample - loss: 0.0074 - val_loss: 0.0076
Epoch 5/100
21988/21988 [==============================] - 0s 20us/sample - loss: 0.0069 - val_loss: 0.0070
Epoch 6/100
21988/21988 [==============================] - 0s 17us/sample - loss: 0.0064 - val_loss: 0.0065
Epoch 7/100
21988/21988 [==============================] - 0s 17us/sample - loss: 0.0059 - val_loss: 0.0060
Epoch 8/100
21988/21988 [==============================] - 0s 18us/sample - loss: 0.0055 - val_loss: 0.0056
Epoch 9/100
21988/21988 [==============================] - 0s 18us/sample - loss: 0.005

Epoch 73/100
21988/21988 [==============================] - 0s 15us/sample - loss: 3.7548e-04 - val_loss: 4.1255e-04
Epoch 74/100
21988/21988 [==============================] - 0s 14us/sample - loss: 3.7208e-04 - val_loss: 3.9508e-04
Epoch 75/100
21988/21988 [==============================] - 0s 14us/sample - loss: 3.6641e-04 - val_loss: 3.8991e-04
Epoch 76/100
21988/21988 [==============================] - 0s 13us/sample - loss: 3.6245e-04 - val_loss: 3.8631e-04
Epoch 77/100
21988/21988 [==============================] - 0s 14us/sample - loss: 3.5795e-04 - val_loss: 3.9327e-04
Epoch 78/100
21988/21988 [==============================] - 0s 14us/sample - loss: 3.5431e-04 - val_loss: 3.7495e-04
Epoch 79/100
21988/21988 [==============================] - 0s 14us/sample - loss: 3.5006e-04 - val_loss: 3.8441e-04
Epoch 80/100
21988/21988 [==============================] - 0s 14us/sample - loss: 3.4168e-04 - val_loss: 3.7065e-04
Epoch 81/100
21988/21988 [==============================] - 0s 1

(4.2333817, array([[ 2.04760044,  2.04787703,  2.02131796],
        [ 0.04591645,  0.04605474,  0.04422323],
        [-0.07633646, -0.08975109, -0.08061689],
        [ 0.26960885,  0.28094905,  0.27296507],
        [-0.95298941, -0.95222879, -0.96015489],
        [ 0.05366098,  0.05538967,  0.05213825],
        [-1.0759338 , -1.07399767, -1.07982481],
        [-1.76450535, -1.7786806 , -1.72436023],
        [-0.43230818, -0.43265391, -0.44205743],
        [-1.61044455, -1.60152451, -1.58438289]]))

In [135]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=100,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 17us/sample - loss: 2.8348e-04 - val_loss: 3.2076e-04
Epoch 2/100
21988/21988 [==============================] - 0s 20us/sample - loss: 2.8142e-04 - val_loss: 3.1725e-04
Epoch 3/100
21988/21988 [==============================] - 0s 19us/sample - loss: 2.7655e-04 - val_loss: 3.1016e-04
Epoch 4/100
21988/21988 [==============================] - 0s 15us/sample - loss: 2.7439e-04 - val_loss: 3.0338e-04
Epoch 5/100
21988/21988 [==============================] - 0s 15us/sample - loss: 2.7292e-04 - val_loss: 2.9655e-04
Epoch 6/100
21988/21988 [==============================] - 0s 15us/sample - loss: 2.7419e-04 - val_loss: 3.0129e-04
Epoch 7/100
21988/21988 [==============================] - 0s 15us/sample - loss: 2.7296e-04 - val_loss: 3.2677e-04
Epoch 8/100
21988/21988 [==============================] - 0s 14us/sample - loss: 2.6859e-04 - val_loss: 2.8873e-04
Epoch 9/100
21988/21988

21988/21988 [==============================] - 0s 15us/sample - loss: 2.0552e-04 - val_loss: 2.3026e-04
Epoch 71/100
21988/21988 [==============================] - 0s 14us/sample - loss: 2.0746e-04 - val_loss: 2.3834e-04
Epoch 72/100
21988/21988 [==============================] - 0s 14us/sample - loss: 2.0781e-04 - val_loss: 2.4091e-04
Epoch 73/100
21988/21988 [==============================] - 0s 14us/sample - loss: 2.0483e-04 - val_loss: 2.2080e-04
Epoch 74/100
21988/21988 [==============================] - 0s 15us/sample - loss: 2.0407e-04 - val_loss: 2.2679e-04
Epoch 75/100
21988/21988 [==============================] - 0s 15us/sample - loss: 2.0204e-04 - val_loss: 2.1690e-04
Epoch 76/100
21988/21988 [==============================] - 0s 16us/sample - loss: 2.0441e-04 - val_loss: 2.2023e-04
Epoch 77/100
21988/21988 [==============================] - 0s 15us/sample - loss: 2.0299e-04 - val_loss: 2.2708e-04
Epoch 78/100
21988/21988 [==============================] - 0s 15us/sample - 

(4.287142, array([[ 2.04760044,  2.04787703,  2.03531027],
        [ 0.04591645,  0.04605474,  0.04492377],
        [-0.07633646, -0.08975109, -0.0792871 ],
        [ 0.26960885,  0.28094905,  0.27240422],
        [-0.95298941, -0.95222879, -0.95984888],
        [ 0.05366098,  0.05538967,  0.05279662],
        [-1.0759338 , -1.07399767, -1.08094716],
        [-1.76450535, -1.7786806 , -1.7421335 ],
        [-0.43230818, -0.43265391, -0.43962806],
        [-1.61044455, -1.60152451, -1.59703398]]))

In [136]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=100,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 15us/sample - loss: 1.9440e-04 - val_loss: 2.0890e-04
Epoch 2/100
21988/21988 [==============================] - 0s 17us/sample - loss: 1.9451e-04 - val_loss: 2.1635e-04
Epoch 3/100
21988/21988 [==============================] - 0s 17us/sample - loss: 1.9208e-04 - val_loss: 2.1968e-04
Epoch 4/100
21988/21988 [==============================] - 0s 16us/sample - loss: 1.9399e-04 - val_loss: 2.0704e-04
Epoch 5/100
21988/21988 [==============================] - 0s 17us/sample - loss: 1.9101e-04 - val_loss: 2.0395e-04
Epoch 6/100
21988/21988 [==============================] - 0s 16us/sample - loss: 1.9056e-04 - val_loss: 2.1402e-04
Epoch 7/100
21988/21988 [==============================] - 0s 15us/sample - loss: 1.8877e-04 - val_loss: 2.0894e-04
Epoch 8/100
21988/21988 [==============================] - 0s 17us/sample - loss: 1.8860e-04 - val_loss: 2.0343e-04
Epoch 9/100
21988/21988

21988/21988 [==============================] - 0s 18us/sample - loss: 1.7505e-04 - val_loss: 1.8696e-04
Epoch 71/100
21988/21988 [==============================] - 0s 17us/sample - loss: 1.7491e-04 - val_loss: 1.8706e-04
Epoch 72/100
21988/21988 [==============================] - 0s 16us/sample - loss: 1.7373e-04 - val_loss: 1.8614e-04
Epoch 73/100
21988/21988 [==============================] - 0s 17us/sample - loss: 1.7438e-04 - val_loss: 1.9833e-04
Epoch 74/100
21988/21988 [==============================] - 0s 15us/sample - loss: 1.7573e-04 - val_loss: 1.9629e-04
Epoch 75/100
21988/21988 [==============================] - 0s 14us/sample - loss: 1.7398e-04 - val_loss: 1.8978e-04
Epoch 76/100
21988/21988 [==============================] - 0s 14us/sample - loss: 1.7339e-04 - val_loss: 1.9457e-04
Epoch 77/100
21988/21988 [==============================] - 0s 16us/sample - loss: 1.7402e-04 - val_loss: 1.8535e-04
Epoch 78/100
21988/21988 [==============================] - 0s 16us/sample - 

(4.30516, array([[ 2.04760044,  2.04787703,  2.03771806],
        [ 0.04591645,  0.04605474,  0.04317676],
        [-0.07633646, -0.08975109, -0.08076099],
        [ 0.26960885,  0.28094905,  0.27012107],
        [-0.95298941, -0.95222879, -0.9613049 ],
        [ 0.05366098,  0.05538967,  0.0510315 ],
        [-1.0759338 , -1.07399767, -1.0828588 ],
        [-1.76450535, -1.7786806 , -1.74977708],
        [-0.43230818, -0.43265391, -0.44056034],
        [-1.61044455, -1.60152451, -1.60287404]]))

In [137]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=200,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6843e-04 - val_loss: 1.8138e-04
Epoch 2/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6811e-04 - val_loss: 1.8266e-04
Epoch 3/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6575e-04 - val_loss: 1.8245e-04
Epoch 4/100
21988/21988 [==============================] - 0s 9us/sample - loss: 1.6657e-04 - val_loss: 1.8476e-04
Epoch 5/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6735e-04 - val_loss: 1.8719e-04
Epoch 6/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6617e-04 - val_loss: 1.8129e-04
Epoch 7/100
21988/21988 [==============================] - 0s 9us/sample - loss: 1.6629e-04 - val_loss: 1.8793e-04
Epoch 8/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6604e-04 - val_loss: 1.8121e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 8us/sample - loss: 1.6309e-04 - val_loss: 1.8044e-04
Epoch 72/100
21988/21988 [==============================] - 0s 9us/sample - loss: 1.6123e-04 - val_loss: 1.7531e-04
Epoch 73/100
21988/21988 [==============================] - 0s 9us/sample - loss: 1.6247e-04 - val_loss: 1.7564e-04
Epoch 74/100
21988/21988 [==============================] - 0s 9us/sample - loss: 1.6229e-04 - val_loss: 1.8339e-04
Epoch 75/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6144e-04 - val_loss: 1.7760e-04
Epoch 76/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6145e-04 - val_loss: 1.7769e-04
Epoch 77/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6300e-04 - val_loss: 1.7776e-04
Epoch 78/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6281e-04 - val_loss: 1.8046e-04
Epoch 79/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6

(4.309318, array([[ 2.04760044,  2.04787703,  2.0391202 ],
        [ 0.04591645,  0.04605474,  0.04438864],
        [-0.07633646, -0.08975109, -0.07936376],
        [ 0.26960885,  0.28094905,  0.27097717],
        [-0.95298941, -0.95222879, -0.9593609 ],
        [ 0.05366098,  0.05538967,  0.0522313 ],
        [-1.0759338 , -1.07399767, -1.08102036],
        [-1.76450535, -1.7786806 , -1.74980021],
        [-0.43230818, -0.43265391, -0.43872732],
        [-1.61044455, -1.60152451, -1.6022718 ]]))

In [138]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=200,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6272e-04 - val_loss: 1.8114e-04
Epoch 2/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6012e-04 - val_loss: 1.7764e-04
Epoch 3/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6083e-04 - val_loss: 1.7671e-04
Epoch 4/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6276e-04 - val_loss: 1.8436e-04
Epoch 5/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.5926e-04 - val_loss: 1.7735e-04
Epoch 6/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.5962e-04 - val_loss: 1.7401e-04
Epoch 7/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.6121e-04 - val_loss: 1.8005e-04
Epoch 8/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.6193e-04 - val_loss: 1.7934e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 8us/sample - loss: 1.5712e-04 - val_loss: 1.7135e-04
Epoch 72/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.5788e-04 - val_loss: 1.7437e-04
Epoch 73/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5544e-04 - val_loss: 1.7009e-04
Epoch 74/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5996e-04 - val_loss: 1.6927e-04
Epoch 75/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.5533e-04 - val_loss: 1.7180e-04
Epoch 76/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.5654e-04 - val_loss: 1.7690e-04
Epoch 77/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.5636e-04 - val_loss: 1.7953e-04
Epoch 78/100
21988/21988 [==============================] - 0s 7us/sample - loss: 1.5782e-04 - val_loss: 1.7702e-04
Epoch 79/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5

(4.306518, array([[ 2.04760044,  2.04787703,  2.03758407],
        [ 0.04591645,  0.04605474,  0.04548238],
        [-0.07633646, -0.08975109, -0.07795243],
        [ 0.26960885,  0.28094905,  0.27147722],
        [-0.95298941, -0.95222879, -0.95628303],
        [ 0.05366098,  0.05538967,  0.05330469],
        [-1.0759338 , -1.07399767, -1.07785189],
        [-1.76450535, -1.7786806 , -1.74712741],
        [-0.43230818, -0.43265391, -0.436472  ],
        [-1.61044455, -1.60152451, -1.59932268]]))

In [139]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=200,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 9us/sample - loss: 1.5761e-04 - val_loss: 1.6931e-04
Epoch 2/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5665e-04 - val_loss: 1.7736e-04
Epoch 3/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5561e-04 - val_loss: 1.6904e-04
Epoch 4/100
21988/21988 [==============================] - 0s 9us/sample - loss: 1.5695e-04 - val_loss: 1.7238e-04
Epoch 5/100
21988/21988 [==============================] - 0s 10us/sample - loss: 1.5696e-04 - val_loss: 1.7088e-04
Epoch 6/100
21988/21988 [==============================] - 0s 10us/sample - loss: 1.5512e-04 - val_loss: 1.7348e-04
Epoch 7/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5805e-04 - val_loss: 1.6983e-04
Epoch 8/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5563e-04 - val_loss: 1.7092e-04
Epoch 9/100
21988/21988 [====

21988/21988 [==============================] - 0s 9us/sample - loss: 1.5461e-04 - val_loss: 1.6600e-04
Epoch 72/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5221e-04 - val_loss: 1.6513e-04
Epoch 73/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5223e-04 - val_loss: 1.7361e-04
Epoch 74/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5481e-04 - val_loss: 1.6938e-04
Epoch 75/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5143e-04 - val_loss: 1.7340e-04
Epoch 76/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5625e-04 - val_loss: 1.6629e-04
Epoch 77/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5284e-04 - val_loss: 1.7719e-04
Epoch 78/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5354e-04 - val_loss: 1.6873e-04
Epoch 79/100
21988/21988 [==============================] - 0s 8us/sample - loss: 1.5

(4.3149953, array([[ 2.04760044,  2.04787703,  2.04033303],
        [ 0.04591645,  0.04605474,  0.04524034],
        [-0.07633646, -0.08975109, -0.07826725],
        [ 0.26960885,  0.28094905,  0.2713604 ],
        [-0.95298941, -0.95222879, -0.9575305 ],
        [ 0.05366098,  0.05538967,  0.05306707],
        [-1.0759338 , -1.07399767, -1.07932734],
        [-1.76450535, -1.7786806 , -1.75056565],
        [-0.43230818, -0.43265391, -0.43705252],
        [-1.61044455, -1.60152451, -1.6022085 ]]))

In [140]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4862e-04 - val_loss: 1.6401e-04
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4788e-04 - val_loss: 1.6513e-04
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4775e-04 - val_loss: 1.6442e-04
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4815e-04 - val_loss: 1.6417e-04
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4796e-04 - val_loss: 1.6477e-04
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4781e-04 - val_loss: 1.6403e-04
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4758e-04 - val_loss: 1.6427e-04
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4768e-04 - val_loss: 1.6419e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 1us/sample - loss: 1.4747e-04 - val_loss: 1.6430e-04
Epoch 72/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4750e-04 - val_loss: 1.6399e-04
Epoch 73/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4766e-04 - val_loss: 1.6404e-04
Epoch 74/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4764e-04 - val_loss: 1.6405e-04
Epoch 75/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4753e-04 - val_loss: 1.6398e-04
Epoch 76/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4759e-04 - val_loss: 1.6440e-04
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4755e-04 - val_loss: 1.6440e-04
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4794e-04 - val_loss: 1.6390e-04
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4

(4.310351, array([[ 2.04760044,  2.04787703,  2.03903103],
        [ 0.04591645,  0.04605474,  0.04617141],
        [-0.07633646, -0.08975109, -0.07719344],
        [ 0.26960885,  0.28094905,  0.2720288 ],
        [-0.95298941, -0.95222879, -0.9554745 ],
        [ 0.05366098,  0.05538967,  0.05398908],
        [-1.0759338 , -1.07399767, -1.0771426 ],
        [-1.76450535, -1.7786806 , -1.74772012],
        [-0.43230818, -0.43265391, -0.43556905],
        [-1.61044455, -1.60152451, -1.59950078]]))

In [142]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4776e-04 - val_loss: 1.6436e-04
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4805e-04 - val_loss: 1.6387e-04
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4752e-04 - val_loss: 1.6404e-04
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4752e-04 - val_loss: 1.6422e-04
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4756e-04 - val_loss: 1.6389e-04
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4767e-04 - val_loss: 1.6403e-04
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4755e-04 - val_loss: 1.6470e-04
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4757e-04 - val_loss: 1.6422e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 1us/sample - loss: 1.4727e-04 - val_loss: 1.6381e-04
Epoch 72/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4725e-04 - val_loss: 1.6399e-04
Epoch 73/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4731e-04 - val_loss: 1.6382e-04
Epoch 74/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4728e-04 - val_loss: 1.6391e-04
Epoch 75/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4744e-04 - val_loss: 1.6383e-04
Epoch 76/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4739e-04 - val_loss: 1.6378e-04
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4738e-04 - val_loss: 1.6408e-04
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4772e-04 - val_loss: 1.6380e-04
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4

(4.3122716, array([[ 2.04760044,  2.04787703,  2.03844261],
        [ 0.04591645,  0.04605474,  0.04466283],
        [-0.07633646, -0.08975109, -0.07875222],
        [ 0.26960885,  0.28094905,  0.27061343],
        [-0.95298941, -0.95222879, -0.95738399],
        [ 0.05366098,  0.05538967,  0.05248369],
        [-1.0759338 , -1.07399767, -1.07910037],
        [-1.76450535, -1.7786806 , -1.74994981],
        [-0.43230818, -0.43265391, -0.43727177],
        [-1.61044455, -1.60152451, -1.60166943]]))

In [143]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4771e-04 - val_loss: 1.6428e-04
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4783e-04 - val_loss: 1.6416e-04
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4779e-04 - val_loss: 1.6377e-04
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4748e-04 - val_loss: 1.6417e-04
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4764e-04 - val_loss: 1.6376e-04
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4752e-04 - val_loss: 1.6385e-04
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4748e-04 - val_loss: 1.6396e-04
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4723e-04 - val_loss: 1.6431e-04
Epoch 9/100
21988/21988 [======

21988/21988 [==============================] - 0s 1us/sample - loss: 1.4715e-04 - val_loss: 1.6368e-04
Epoch 72/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4715e-04 - val_loss: 1.6367e-04
Epoch 73/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4731e-04 - val_loss: 1.6380e-04
Epoch 74/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4722e-04 - val_loss: 1.6384e-04
Epoch 75/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4714e-04 - val_loss: 1.6366e-04
Epoch 76/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4727e-04 - val_loss: 1.6373e-04
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4714e-04 - val_loss: 1.6368e-04
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4715e-04 - val_loss: 1.6368e-04
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4

(4.308916, array([[ 2.04760044,  2.04787703,  2.03796315],
        [ 0.04591645,  0.04605474,  0.04582721],
        [-0.07633646, -0.08975109, -0.07748221],
        [ 0.26960885,  0.28094905,  0.27158296],
        [-0.95298941, -0.95222879, -0.95539802],
        [ 0.05366098,  0.05538967,  0.05364136],
        [-1.0759338 , -1.07399767, -1.07702303],
        [-1.76450535, -1.7786806 , -1.74741769],
        [-0.43230818, -0.43265391, -0.4357003 ],
        [-1.61044455, -1.60152451, -1.59922957]]))

In [144]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=n,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4752e-04 - val_loss: 1.6435e-04
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4761e-04 - val_loss: 1.6376e-04
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4722e-04 - val_loss: 1.6366e-04
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4729e-04 - val_loss: 1.6389e-04
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4754e-04 - val_loss: 1.6374e-04
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4726e-04 - val_loss: 1.6378e-04
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4713e-04 - val_loss: 1.6415e-04
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4740e-04 - val_loss: 1.6402e-04
Epoch 9/100
21988/21988 [======

Epoch 72/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6362e-04
Epoch 73/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6362e-04
Epoch 74/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6361e-04
Epoch 75/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6361e-04
Epoch 76/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6361e-04
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6361e-04
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6362e-04
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4706e-04 - val_loss: 1.6362e-04
Epoch 80/100
21988/21988 [==============================] - 0s 1us/sampl

(4.311177, array([[ 2.04760044,  2.04787703,  2.03865719],
        [ 0.04591645,  0.04605474,  0.04546428],
        [-0.07633646, -0.08975109, -0.07790995],
        [ 0.26960885,  0.28094905,  0.27133891],
        [-0.95298941, -0.95222879, -0.95628321],
        [ 0.05366098,  0.05538967,  0.05328251],
        [-1.0759338 , -1.07399767, -1.0779711 ],
        [-1.76450535, -1.7786806 , -1.74870849],
        [-0.43230818, -0.43265391, -0.43631572],
        [-1.61044455, -1.60152451, -1.60044491]]))

In [145]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=n,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4705e-04 - val_loss: 1.6360e-04
Epoch 2/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4705e-04 - val_loss: 1.6360e-04
Epoch 3/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4705e-04 - val_loss: 1.6360e-04
Epoch 4/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4705e-04 - val_loss: 1.6360e-04
Epoch 5/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4704e-04 - val_loss: 1.6360e-04
Epoch 6/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4704e-04 - val_loss: 1.6360e-04
Epoch 7/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4704e-04 - val_loss: 1.6360e-04
Epoch 8/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4704e-04 - val_loss: 1.6360e-04
Epoch 9/100
21988/21988 [======

Epoch 72/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6357e-04
Epoch 73/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6357e-04
Epoch 74/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6357e-04
Epoch 75/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6356e-04
Epoch 76/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6356e-04
Epoch 77/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6356e-04
Epoch 78/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6356e-04
Epoch 79/100
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4701e-04 - val_loss: 1.6356e-04
Epoch 80/100
21988/21988 [==============================] - 0s 1us/sampl

(4.3112354, array([[ 2.04760044,  2.04787703,  2.03867483],
        [ 0.04591645,  0.04605474,  0.04546845],
        [-0.07633646, -0.08975109, -0.07790503],
        [ 0.26960885,  0.28094905,  0.27134165],
        [-0.95298941, -0.95222879, -0.95627916],
        [ 0.05366098,  0.05538967,  0.05328666],
        [-1.0759338 , -1.07399767, -1.07796836],
        [-1.76450535, -1.7786806 , -1.74872398],
        [-0.43230818, -0.43265391, -0.4363094 ],
        [-1.61044455, -1.60152451, -1.60045493]]))

In [146]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=n,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 21988 samples, validate on 2444 samples
Epoch 1/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 2/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 3/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 4/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 5/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 6/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 7/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 8/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4700e-04 - val_loss: 1.6355e-04
Epoch 9/1000
21988/2198

Epoch 71/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 72/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 73/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 74/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 75/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 76/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 77/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 78/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4696e-04 - val_loss: 1.6351e-04
Epoch 79/1000
21988/21988 [==============================] - 0s 

Epoch 141/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4693e-04 - val_loss: 1.6347e-04
Epoch 142/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4693e-04 - val_loss: 1.6347e-04
Epoch 143/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4693e-04 - val_loss: 1.6347e-04
Epoch 144/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4693e-04 - val_loss: 1.6347e-04
Epoch 145/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4693e-04 - val_loss: 1.6347e-04
Epoch 146/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4693e-04 - val_loss: 1.6347e-04
Epoch 147/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4693e-04 - val_loss: 1.6347e-04
Epoch 148/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4692e-04 - val_loss: 1.6347e-04
Epoch 149/1000
21988/21988 [============================

Epoch 211/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6344e-04
Epoch 212/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6343e-04
Epoch 213/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6343e-04
Epoch 214/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6343e-04
Epoch 215/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6343e-04
Epoch 216/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6343e-04
Epoch 217/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6343e-04
Epoch 218/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4689e-04 - val_loss: 1.6343e-04
Epoch 219/1000
21988/21988 [============================

Epoch 281/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 282/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 283/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 284/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 285/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 286/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 287/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 288/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4685e-04 - val_loss: 1.6339e-04
Epoch 289/1000
21988/21988 [============================

...

Epoch 771/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6307e-04
Epoch 772/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6307e-04
Epoch 773/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6307e-04
Epoch 774/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6306e-04
Epoch 775/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6306e-04
Epoch 776/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6306e-04
Epoch 777/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6306e-04
Epoch 778/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4655e-04 - val_loss: 1.6306e-04
Epoch 779/1000
21988/21988 [============================

Epoch 841/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 842/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 843/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 844/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 845/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 846/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 847/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 848/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4650e-04 - val_loss: 1.6301e-04
Epoch 849/1000
21988/21988 [============================

Epoch 911/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6296e-04
Epoch 912/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6296e-04
Epoch 913/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6296e-04
Epoch 914/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6296e-04
Epoch 915/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6296e-04
Epoch 916/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6296e-04
Epoch 917/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6296e-04
Epoch 918/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4645e-04 - val_loss: 1.6295e-04
Epoch 919/1000
21988/21988 [============================

Epoch 981/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 982/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 983/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 984/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 985/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 986/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 987/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 988/1000
21988/21988 [==============================] - 0s 1us/sample - loss: 1.4640e-04 - val_loss: 1.6290e-04
Epoch 989/1000
21988/21988 [============================

(4.3118486, array([[ 2.04760044,  2.04787703,  2.03879285],
        [ 0.04591645,  0.04605474,  0.04547389],
        [-0.07633646, -0.08975109, -0.07788587],
        [ 0.26960885,  0.28094905,  0.27132046],
        [-0.95298941, -0.95222879, -0.95623815],
        [ 0.05366098,  0.05538967,  0.05329117],
        [-1.0759338 , -1.07399767, -1.07794249],
        [-1.76450535, -1.7786806 , -1.74891198],
        [-0.43230818, -0.43265391, -0.43626025],
        [-1.61044455, -1.60152451, -1.60057342]]))

In [152]:
model.fit(x, y, 
          batch_size=n,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, 0]]), 0, 1)
px.max() - px.min(), res

Train on 24432 samples
Epoch 1/1000
24432/24432 [==============================] - 0s 1us/sample - loss: 1.4293e-04
Epoch 2/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4293e-04
Epoch 3/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4293e-04
Epoch 4/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4293e-04
Epoch 5/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4293e-04
Epoch 6/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4293e-04
Epoch 7/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4292e-04
Epoch 8/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4292e-04
Epoch 9/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4292e-04
Epoch 10/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4292e-04
Epoch 11/1000
24432/24432 [===================

24432/24432 [==============================] - 0s 0us/sample - loss: 1.4327e-04
Epoch 89/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4301e-04
Epoch 90/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4292e-04
Epoch 91/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4301e-04
Epoch 92/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4316e-04
Epoch 93/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4322e-04
Epoch 94/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4312e-04
Epoch 95/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4296e-04
Epoch 96/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4286e-04
Epoch 97/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4288e-04
Epoch 98/1000
24432/24432 [==============================] - 0s 0us/sample

Epoch 175/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4273e-04
Epoch 176/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4273e-04
Epoch 177/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4273e-04
Epoch 178/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4273e-04
Epoch 179/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4273e-04
Epoch 180/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4273e-04
Epoch 181/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4272e-04
Epoch 182/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4272e-04
Epoch 183/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4272e-04
Epoch 184/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4272e-04
Epoch 185/1000
24432/24432 [======================

Epoch 262/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4323e-04
Epoch 263/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4305e-04
Epoch 264/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4282e-04
Epoch 265/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4270e-04
Epoch 266/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4270e-04
Epoch 267/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4278e-04
Epoch 268/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4285e-04
Epoch 269/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4284e-04
Epoch 270/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4279e-04
Epoch 271/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4274e-04
Epoch 272/1000
24432/24432 [======================

Epoch 349/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 350/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 351/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 352/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 353/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 354/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 355/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 356/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 357/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4253e-04
Epoch 358/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4252e-04
Epoch 359/1000
24432/24432 [======================

...

Epoch 697/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4221e-04
Epoch 698/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4219e-04
Epoch 699/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4217e-04
Epoch 700/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4216e-04
Epoch 701/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4216e-04
Epoch 702/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4216e-04
Epoch 703/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4216e-04
Epoch 704/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4216e-04
Epoch 705/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4216e-04
Epoch 706/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4216e-04
Epoch 707/1000
24432/24432 [======================

Epoch 784/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 785/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 786/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 787/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 788/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 789/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 790/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 791/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 792/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 793/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4205e-04
Epoch 794/1000
24432/24432 [======================

Epoch 871/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4197e-04
Epoch 872/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4197e-04
Epoch 873/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4197e-04
Epoch 874/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4197e-04
Epoch 875/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4197e-04
Epoch 876/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4198e-04
Epoch 877/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4198e-04
Epoch 878/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4198e-04
Epoch 879/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4198e-04
Epoch 880/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4198e-04
Epoch 881/1000
24432/24432 [======================

Epoch 958/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4187e-04
Epoch 959/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4187e-04
Epoch 960/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 961/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 962/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 963/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 964/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 965/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 966/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 967/1000
24432/24432 [==============================] - 0s 0us/sample - loss: 1.4186e-04
Epoch 968/1000
24432/24432 [======================

(4.3216076, array([[ 2.04760044,  2.04787703,  2.04070544],
        [ 0.04591645,  0.04605474,  0.04482415],
        [-0.07633646, -0.08975109, -0.07848191],
        [ 0.26960885,  0.28094905,  0.27055383],
        [-0.95298941, -0.95222879, -0.95730042],
        [ 0.05366098,  0.05538967,  0.0526377 ],
        [-1.0759338 , -1.07399767, -1.07926893],
        [-1.76450535, -1.7786806 , -1.75311339],
        [-0.43230818, -0.43265391, -0.43681338],
        [-1.61044455, -1.60152451, -1.60390079]]))

In [153]:
model.get_weights()

[array([[0.10768022]], dtype=float32),
 array([[1.]], dtype=float32),
 array([-0.02169234], dtype=float32),
 array([[9.371996]], dtype=float32),
 array([0.20177206], dtype=float32)]

In [147]:
mean((tx[:,-1,0]-ty)**2)

0.00012199883962722794

In [141]:
mean((x[:,-1,0]-y)**2)

0.00012339544539288403

In [ ]:
model = Sequential([
    Input(shape=(window, 1)),
    SimpleRNN(8, input_shape=(window, 1)),
    Dense(1),
])
model.build()
model.compile(loss='mse', optimizer='adam')
model.summary()

In [ ]:
model.fit(x, y, 
          validation_split=0.1,
          batch_size=50,
          epochs=10)

px = model.predict(vx)
px.max() - px.min(), x[:10], y[:10], px[:10]

In [ ]:
tn = int(0.9 * n)
t_x = tx[:tn]
t_y = ty[:tn]
vx = tx[tn:]
vy = ty[tn:]

In [ ]:
mean((vy-px[0])**2)

In [ ]:
n = ty.shape[0]; n

In [ ]:
mean(((ty - tx[:, -1, 0])**2)[:tn])

In [ ]:
mean((ty - tx[:, :, 0])**2)

In [ ]:
!{python} -m pip install -Uq scipy
from scipy.stats import describe

In [ ]:
describe(ty)

In [ ]:
ty.hi

In [ ]:
w = model.get_weights()
[ r.shape for r in w ]